In [5]:
# Create the directory if it doesn't exist
!mkdir ..\data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 87 1330k   87 1167k    0     0  3115k      0 --:--:-- --:--:-- --:--:-- 3129k
100 1330k  100 1330k    0     0  3363k      0 --:--:-- --:--:-- --:--:-- 3385k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1253k  100 1253k    0     0  2195k      0 --:--:-- --:--:-- --:--:-- 2207k


In [8]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [6]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [9]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as PacoTinoco

Initialized MLflow to track repo "PacoTinoco/nyc-taxi-time-prediction"

Repository PacoTinoco/nyc-taxi-time-prediction initialized!

https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/9e991ac0af45417cb6a8c46fbaf1900e', creation_time=1726630091495, experiment_id='0', last_update_time=1726630091495, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [15]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("PacoTinoco/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

## test
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)


Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [16]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [19]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:6.34602                           
[1]	validation-rmse:5.59131                           
[2]	validation-rmse:5.38253                           
[3]	validation-rmse:5.34096                           
[4]	validation-rmse:5.31669                           
[5]	validation-rmse:5.31087                           
[6]	validation-rmse:5.30307                           
[7]	validation-rmse:5.28951                           
[8]	validation-rmse:5.28666                           
[9]	validation-rmse:5.28697                           
[10]	validation-rmse:5.28584                          
[11]	validation-rmse:5.28643                          
[12]	validation-rmse:5.28572                          
[13]	validation-rmse:5.28447                          
[14]	validation-rmse:5.28297                          
[15]	validation-rmse:5.28224                          
[16]	validation-rmse:5.28272                          
[17]	validation-rmse:5.28207                          
[18]	valid

2024/09/20 17:12:56 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/20 17:12:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:12:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/20 17:13:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-pr

[0]	validation-rmse:8.71265                                                    
[1]	validation-rmse:8.35247                                                    
[2]	validation-rmse:8.02808                                                    
[3]	validation-rmse:7.73636                                                    
[4]	validation-rmse:7.47496                                                    
[5]	validation-rmse:7.24131                                                    
[6]	validation-rmse:7.03284                                                    
[7]	validation-rmse:6.84749                                                    
[8]	validation-rmse:6.68250                                                    
[9]	validation-rmse:6.53631                                                    
[10]	validation-rmse:6.40725                                                   
[11]	validation-rmse:6.29303                                                   
[12]	validation-rmse:6.19229            

2024/09/20 17:13:38 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/20 17:13:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:13:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/20 17:13:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-pr

[0]	validation-rmse:7.89004                                                    
[1]	validation-rmse:7.03968                                                    
[2]	validation-rmse:6.46258                                                    
[3]	validation-rmse:6.08060                                                    
[4]	validation-rmse:5.83390                                                    
[5]	validation-rmse:5.67641                                                    
[6]	validation-rmse:5.57585                                                    
[7]	validation-rmse:5.50642                                                    
[8]	validation-rmse:5.46394                                                    
[9]	validation-rmse:5.43338                                                    
[10]	validation-rmse:5.41250                                                   
[11]	validation-rmse:5.39837                                                   
[12]	validation-rmse:5.38582            

...

2024/09/20 17:20:03 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/20 17:20:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:20:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/20 17:20:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-pr

[0]	validation-rmse:8.43130                                                    
[1]	validation-rmse:7.86246                                                    
[2]	validation-rmse:7.39039                                                    
[3]	validation-rmse:7.00104                                                    
[4]	validation-rmse:6.68261                                                    
[5]	validation-rmse:6.42379                                                    
[6]	validation-rmse:6.21505                                                    
[7]	validation-rmse:6.04690                                                    
[8]	validation-rmse:5.91297                                                    
[9]	validation-rmse:5.80652                                                    
[10]	validation-rmse:5.72198                                                   
[11]	validation-rmse:5.65425                                                   
[12]	validation-rmse:5.60080            

2024/09/20 17:20:34 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/20 17:20:34 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:20:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/20 17:20:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-pr

100%|██████████| 10/10 [08:26<00:00, 50.62s/trial, best loss: 5.182616336428882]
[0]	validation-rmse:8.66585
[1]	validation-rmse:8.26497
[2]	validation-rmse:7.90558
[3]	validation-rmse:7.58444
[4]	validation-rmse:7.29747
[5]	validation-rmse:7.04266
[6]	validation-rmse:6.81561
[7]	validation-rmse:6.61422
[8]	validation-rmse:6.43643
[9]	validation-rmse:6.28018
[10]	validation-rmse:6.14268
[11]	validation-rmse:6.02382
[12]	validation-rmse:5.91787
[13]	validation-rmse:5.82600
[14]	validation-rmse:5.74457
[15]	validation-rmse:5.67291
[16]	validation-rmse:5.61022
[17]	validation-rmse:5.55724
[18]	validation-rmse:5.51052
[19]	validation-rmse:5.47105
[20]	validation-rmse:5.43513
[21]	validation-rmse:5.40462
[22]	validation-rmse:5.37791
[23]	validation-rmse:5.35578
[24]	validation-rmse:5.33522
[25]	validation-rmse:5.31814
[26]	validation-rmse:5.30322
[27]	validation-rmse:5.28904
[28]	validation-rmse:5.27761
[29]	validation-rmse:5.26730
[30]	validation-rmse:5.25786
[31]	validation-rmse:5.24976
[

2024/09/20 17:22:03 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/09/20 17:22:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:22:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/09/20 17:22:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/20 17:22:15 INFO mlflow.tracking._tracking_service.client: 🏃 Vie

In [21]:
best_params

{'learning_rate': np.float64(0.14599396824069683),
 'max_depth': 42,
 'min_child_weight': np.float64(1.7855145549554956),
 'reg_alpha': np.float64(0.05020597797620721),
 'reg_lambda': np.float64(0.0036879070748615603),
 'seed': 42,
 'objective': 'reg:squarederror'}

In [20]:
import mlflow

In [21]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2024/09/20 17:24:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 4
Created version '4' of model 'nyc-taxi-model'.


In [22]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1726631483611, current_stage='None', description='The model version 1 was transitioned to champion on 2024-09-20 17:24:54.731946', last_updated_timestamp=1726874694532, name='nyc-taxi-model', run_id='23b66a83aacb4d04a076a7275dfc65e3', run_link='', source='mlflow-artifacts:/9e991ac0af45417cb6a8c46fbaf1900e/23b66a83aacb4d04a076a7275dfc65e3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [23]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

2024/09/20 17:25:06 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.16.2, required: mlflow==2.16.1)
 - scikit-learn (current: 1.5.1, required: scikit-learn==1.5.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


array([17.868977 , 27.244492 ,  7.5599394, ..., 36.802193 , 10.982415 ,
       19.203161 ], dtype=float32)